# Alpha-Vantage Data Engineering  

In [111]:
import os
import io
import json
import requests
from datetime import datetime
import pandas as pd
from pymongo import MongoClient
import time

In [76]:
secrets_path = os.path.join(os.path.expanduser('~'), "git/bakery/bakery/data/config.json")
with open(secrets_path, "rb") as f:
    secrets = json.loads(f.read().decode())
    api_key = secrets["alpha"]["api_key"]
    limit = secrets["alpha"]["limit"]
    endpoint = secrets["alpha"]["endpoint"]
    conn_str = secrets["mongo"]["conn_str"]

## Exploration

### Company overview

In [77]:
params = {
        "function": "OVERVIEW",
        "symbol": "TSLA",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
r.json()

{'Symbol': 'TSLA',
 'AssetType': 'Common Stock',
 'Name': 'Tesla Inc',
 'Description': "Tesla, Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars, battery energy storage from home to grid-scale, solar panels and solar roof tiles, as well as other related products and services. In 2020, Tesla had the highest sales in the plug-in and battery electric passenger car segments, capturing 16% of the plug-in market (which includes plug-in hybrids) and 23% of the battery-electric (purely electric) market. Through its subsidiary Tesla Energy, the company develops and is a major installer of solar photovoltaic energy generation systems in the United States. Tesla Energy is also one of the largest global suppliers of battery energy storage systems, with 3 GWh of battery storage supplied in 2020.",
 'CIK': '1318605',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'MANUFACTURING',
 'In

### Listing status

In [78]:
params = {
        "function": "LISTING_STATUS",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
data = io.BytesIO(r.content)
df = pd.read_csv(data)
df.head()

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,AXS First Priority CLO Bond ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,NaN,Active


### Quotes

In [79]:
params = {
        "function": "GLOBAL_QUOTE",
        "symbol": "TSLA",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
r_json = r.json()
r_json

{'Global Quote': {'01. symbol': 'TSLA',
  '02. open': '209.9900',
  '03. high': '213.1900',
  '04. low': '207.5600',
  '05. price': '212.1900',
  '06. volume': '102260343',
  '07. latest trading day': '2024-01-19',
  '08. previous close': '211.8800',
  '09. change': '0.3100',
  '10. change percent': '0.1463%'}}

### News

In [80]:
params = {
        "function": "NEWS_SENTIMENT",
        "apikey": api_key,
        "topics": "economy_fiscal",
        "time_from": "20240120T1130"
    }
r = requests.get(endpoint, params)
r.json()

{'items': '14',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Interim Budget 2024-25: Govt may target 5.3% fiscal deficit for FY25',
   'url': 'https://www.business-standard.com/budget/news/budget-forecasters-converge-towards-5-3-fiscal-deficit-target-for-fy25-124012100447_1.html',
   'time_published': '20240121T145932',
   'authors': ['Shiva Rajora'],
   'summary': 'The fiscal deficit target for FY25 is likely to be pegged at 5.3 per cent of the gross domestic product ( GDP ) in the upcoming interim budget , suggests an analysis of multiple projections made by economists and forecasting agencies.',
   'banner_image': 'https://bsmedia.business-standard.com/_media/bs/img/article/2023-12/29/full/1703871427-9685.jpg?im=FitAndFill=(826,465)',
   'so

### Income statements

In [81]:
params = {
        "function": "INCOME_STATEMENT",
        "symbol": "TSLA",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
income_statements = r.json()

In [82]:
income_statements["quarterlyReports"][-1]

{'fiscalDateEnding': '2018-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': '1443000000',
 'totalRevenue': '7226000000',
 'costOfRevenue': '5783000000',
 'costofGoodsAndServicesSold': '25289000',
 'operatingIncome': '414000000',
 'sellingGeneralAndAdministrative': '668000000',
 'researchAndDevelopment': '356000000',
 'operatingExpenses': '1029000000',
 'investmentIncomeNet': '7348000',
 'netInterestIncome': '-174723000',
 'interestIncome': '277000',
 'interestExpense': '175000000',
 'nonInterestIncome': '7441796000',
 'otherNonOperatingIncome': '-14000000',
 'depreciation': '313000000',
 'depreciationAndAmortization': '496737000',
 'incomeBeforeTax': '162000000',
 'incomeTaxExpense': '22000000',
 'interestAndDebtExpense': '174723000',
 'netIncomeFromContinuingOperations': '1442900000',
 'comprehensiveIncomeNetOfTax': '123000000',
 'ebit': '337000000',
 'ebitda': '910273000',
 'netIncome': '140000000'}

### IPO calendar

In [83]:
params = {
        "function": "IPO_CALENDAR",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
data = io.BytesIO(r.content)
df = pd.read_csv(data)
df.head()

,symbol,name,ipoDate,priceRangeLow,priceRangeHigh,currency,exchange


In [84]:
params = {
        "function": "EARNINGS_CALENDAR",
        "symbol": "TSLA",
        "horizon": "3month",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
data = io.BytesIO(r.content)
df = pd.read_csv(data)
df.head()

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
0,TSLA,Tesla Inc,2024-01-23,2023-12-31,0.7400,USD
1,TSLA,Tesla Inc,2024-04-17,2024-03-31,0.8008,USD


In [85]:
params = {
        "function": "TREASURY_YIELD",
        "interval": "daily",
        "maturity": "5year",
        "apikey": api_key
    }
r = requests.get(endpoint, params)
r.json()

{'name': '5-Year Treasury Constant Maturity Rate',
 'interval': 'daily',
 'unit': 'percent',
 'data': [{'date': '2024-01-18', 'value': '4.04'},
  {'date': '2024-01-17', 'value': '4.02'},
  {'date': '2024-01-16', 'value': '3.95'},
  {'date': '2024-01-15', 'value': '.'},
  {'date': '2024-01-12', 'value': '3.84'},
  {'date': '2024-01-11', 'value': '3.90'},
  {'date': '2024-01-10', 'value': '3.99'},
  {'date': '2024-01-09', 'value': '3.97'},
  {'date': '2024-01-08', 'value': '3.97'},
  {'date': '2024-01-05', 'value': '4.02'},
  {'date': '2024-01-04', 'value': '3.97'},
  {'date': '2024-01-03', 'value': '3.90'},
  {'date': '2024-01-02', 'value': '3.93'},
  {'date': '2024-01-01', 'value': '.'},
  {'date': '2023-12-29', 'value': '3.84'},
  {'date': '2023-12-28', 'value': '3.83'},
  {'date': '2023-12-27', 'value': '3.78'},
  {'date': '2023-12-26', 'value': '3.89'},
  {'date': '2023-12-25', 'value': '.'},
  {'date': '2023-12-22', 'value': '3.87'},
  {'date': '2023-12-21', 'value': '3.87'},
  {'d

### Time series

In [86]:
year_month = str(2024) + "-" + f"{1:02}"
interval = "Daily"
symbol = "TSLA"

params = {
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": symbol,
    # "interval": interval,
    # "extended_hours": "true",
    "outputsize": "full",
    # "month": year_month,
    "apikey": api_key
}
r = requests.get(endpoint, params)
r_dict = r.json()

ts_meta = r_dict["Meta Data"]
ts = r_dict[f"Time Series ({interval})"]

In [87]:
ts_meta

{'1. Information': 'Daily Time Series with Splits and Dividend Events',
 '2. Symbol': 'TSLA',
 '3. Last Refreshed': '2024-01-19',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [88]:
ts

{'2024-01-19': {'1. open': '209.99',
  '2. high': '213.19',
  '3. low': '207.56',
  '4. close': '212.19',
  '5. adjusted close': '212.19',
  '6. volume': '102260343',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2024-01-18': {'1. open': '216.88',
  '2. high': '217.45',
  '3. low': '208.74',
  '4. close': '211.88',
  '5. adjusted close': '211.88',
  '6. volume': '108595431',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2024-01-17': {'1. open': '214.86',
  '2. high': '215.67',
  '3. low': '212.01',
  '4. close': '215.55',
  '5. adjusted close': '215.55',
  '6. volume': '103164400',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2024-01-16': {'1. open': '215.1',
  '2. high': '223.49',
  '3. low': '212.18',
  '4. close': '219.91',
  '5. adjusted close': '219.91',
  '6. volume': '115355046',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2024-01-12': {'1. open': '220.08',
  '2. high': '225.34',
 

In [89]:
df = pd.DataFrame(ts).T
df["dttm"] = df.index
df = df.reset_index(drop=True)
df["symbol"] = symbol

df = df.rename(columns={"1. open": "open", 
                        "2. high": "high", 
                        "3. low": "low", 
                        "4. close": "close", 
                        "5. adjusted close": "adj_close", 
                        "6. volume": "volume",
                        "7. dividend amount": "div_amt",
                        "8. split coefficient": "split_coef"})

float_cols = ["open", "high", "low", "close", "adj_close", "volume", "div_amt", "split_coef"]
df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce")
df["dttm"] = pd.to_datetime(df["dttm"])
df

,open,high,low,close,adj_close,volume,div_amt,split_coef,dttm,symbol
0,209.99,213.1900,207.5600,212.19,212.190000,102260343,0.0,1.0,2024-01-19,TSLA
1,216.88,217.4500,208.7400,211.88,211.880000,108595431,0.0,1.0,2024-01-18,TSLA
2,214.86,215.6700,212.0100,215.55,215.550000,103164400,0.0,1.0,2024-01-17,TSLA
3,215.10,223.4900,212.1800,219.91,219.910000,115355046,0.0,1.0,2024-01-16,TSLA
4,220.08,225.3400,217.1501,218.89,218.890000,123043812,0.0,1.0,2024-01-12,TSLA
...,...,...,...,...,...,...,...,...,...,...
3408,20.00,20.0000,15.8300,16.11,1.074000,6866900,0.0,1.0,2010-07-06,TSLA
3409,23.00,23.1000,18.7100,19.20,1.280000,5139800,0.0,1.0,2010-07-02,TSLA
3410,25.00,25.9200,20.2700,21.96,1.464000,8218800,0.0,1.0,2010-07-01,TSLA
3411,25.79,30.4192,23.3000,23.83,1.588667,17187100,0.0,1.0,2010-06-30,TSLA


## ETL functions
### Active list  

In [91]:
def extract_active_list(endpoint, api_key):
    """

    """

    params = {
        "function": "LISTING_STATUS",
        "apikey": api_key
    }

    return requests.get(endpoint, params)

def transform_active_list(response):
    """

    """

    data = io.BytesIO(response.content)
    df = pd.read_csv(data)
    df.drop(columns="delistingDate", inplace=True)
    df.rename(columns={"assetType": "asset_type", "ipoDate": "ipo_dt"}, inplace=True)
    df["ipo_dt"] = pd.to_datetime(df["ipo_dt"])
    df["refresh_dttm"]= datetime.today()

    return df

def load_active_list(df, client):
    """ 
    
    """

    rows = df.to_dict("records")
    coll = client["bakery"]["alpha_active_list"]
    result = coll.insert_many(rows)
    
    return result

### Time Series  
#### Daily

In [110]:
def extract_alpha_daily(symbol, api_key, endpoint):
    """ 
    
    """

    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "outputsize": "full",
        "datatype": "json",
        "apikey": api_key
    }

    try:
        r = requests.get(endpoint, params)
        r_dict = r.json()["Time Series (Daily)"]
    except requests.exceptions.RequestException as re:
        print(f"Request error for symbol {symbol}\n{re}")
    except KeyError as ke:
        print(f"Key error for symbol {symbol}\n{ke}\nAvailable keys: {r.json().keys()}, likely caused by rate limiting.")
    
    return r_dict

def transform_alpha_daily(symbol, data):
    """ 
    
    """

    df = pd.DataFrame(data).T
    df["dttm"] = df.index
    df = df.reset_index(drop=True)
    df["symbol"] = symbol

    df = df.rename(columns={"1. open": "open", 
                            "2. high": "high", 
                            "3. low": "low", 
                            "4. close": "close", 
                            "5. adjusted close": "adj_close", 
                            "6. volume": "volume",
                            "7. dividend amount": "div_amt",
                            "8. split coefficient": "split_coef"})

    float_cols = ["open", "high", "low", "close", "adj_close", "volume", "div_amt", "split_coef"]
    df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce")
    df["dttm"] = pd.to_datetime(df["dttm"])
    
    return df

def load_alpha_daily(df, client):
    """ 
    
    """

    rows = df.to_dict("records")
    coll = client["bakery"]["alpha_stock_daily"]
    result = coll.insert_many(rows)

    return result

In [2]:
def extract_alpha_hourly(symbol, year, month, api_key, endpoint):
    """ 
    
    """
    
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": "60min",
        "extended_hours": "true",
        "outputsize": "full",
        "month": str(year) + "-" + f"{month:02}",
        "apikey": api_key
    }
    
    try:
        r = requests.get(endpoint, params)
        r_dict = r.json()["Time Series (60min)"]
    except requests.exceptions.RequestException as re:
        print(f"Request error for symbol {symbol} and year-month {year_month}\n{re}")
    except KeyError as ke:
        print(f"Key error for symbol {symbol} and year-month {year_month}\n{ke}\nAvailable keys: {r.json().keys()}, likely caused by rate limiting.")
    
    return r_dict

def transform_alpha_hourly(symbol, data):
    """ 
    
    """

    df = pd.DataFrame(data).T
    df["dttm"] = df.index
    df = df.reset_index(drop=True)
    df["symbol"] = symbol

    df = df.rename(columns={"1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"})

    float_cols = ["open", "high", "low", "close"]
    df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce")
    df["volume"] = pd.to_numeric(df["volume"])
    df["dttm"] = pd.to_datetime(df["dttm"])
    
    return df

def load_alpha_hourly(client, df):
    """ 
    
    """

    rows = df.to_dict("records")
    coll = client["bakery"]["alpha_stock_intraday_hourly"]
    result = coll.insert_many(rows)

    return result


## Tests

In [3]:
client = MongoClient(conn_str)

### Active list

In [92]:
r = extract_active_list(endpoint, api_key)
df = transform_active_list(r)
df.head()

,symbol,name,exchange,asset_type,ipo_dt,status,refresh_dttm
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,Active,2024-01-21 17:07:56.228186
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,Active,2024-01-21 17:07:56.228186
2,AAA,AXS First Priority CLO Bond ETF,NYSE ARCA,ETF,2020-09-09,Active,2024-01-21 17:07:56.228186
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,Active,2024-01-21 17:07:56.228186
4,AACG,ATA Creativity Global,NASDAQ,Stock,2008-01-29,Active,2024-01-21 17:07:56.228186


In [93]:
result = load_active_list(df, client)

### Time Series  
#### Daily

In [100]:
data = extract_alpha_daily(symbol, api_key, endpoint)

In [103]:
df = transform_alpha_daily(symbol, data)
df.head()

,open,high,low,close,adj_close,volume,div_amt,split_coef,dttm,symbol
0,209.99,213.19,207.5600,212.19,212.19,102260343,0.0,1.0,2024-01-19,TSLA
1,216.88,217.45,208.7400,211.88,211.88,108595431,0.0,1.0,2024-01-18,TSLA
2,214.86,215.67,212.0100,215.55,215.55,103164400,0.0,1.0,2024-01-17,TSLA
3,215.10,223.49,212.1800,219.91,219.91,115355046,0.0,1.0,2024-01-16,TSLA
4,220.08,225.34,217.1501,218.89,218.89,123043812,0.0,1.0,2024-01-12,TSLA


In [105]:
result = load_alpha_daily(df, client)

#### Hourly

In [106]:
year, month = 2024, 1

data = extract_alpha_hourly(symbol, year, month, api_key, endpoint)

In [109]:
df = transform_alpha_hourly(symbol, data)
df.head()

,open,high,low,close,volume,dttm,symbol
0,212.22,212.460,212.050,212.40,100174,2024-01-19 19:00:00,TSLA
1,212.32,212.420,211.980,212.21,74769,2024-01-19 18:00:00,TSLA
2,212.38,291.308,178.312,212.33,386282,2024-01-19 17:00:00,TSLA
3,212.13,240.450,210.276,212.35,4896408,2024-01-19 16:00:00,TSLA
4,211.96,212.820,210.691,212.17,13964735,2024-01-19 15:00:00,TSLA


In [108]:
result = load_alpha_hourly(client, df)

### Rate Limiting

In [113]:
symbols = ["TSLA", "MSFT", "NVDA", "AMZN", "AAPL"]
total_requests = 0
t0 = time.time()

for symbol in symbols:
    for year in range(2020, 2024):
        for month in range(1, 13):
            load_alpha_hourly(client, transform_alpha_hourly(symbol, extract_alpha_hourly(symbol, year, month, api_key, endpoint)))
            total_requests += 1
            total_seconds = time.time() - t0
            requests_per_minute = total_requests / (total_seconds / 60)
            while (requests_per_minute > limit):
                time.sleep(1)
                total_seconds = time.time() - t0
                requests_per_minute = total_requests / (total_seconds / 60)